Tiering of BloodHound data

Import libs

In [2]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [3]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="bloodhound")
relpath = pathlib.Path().absolute().as_uri()

Tiering functions

In [4]:
tier_ous = '''
UNWIND value.ous as o
MATCH (obj) WHERE (obj.distinguishedname ENDS WITH o.path) AND ((obj:User) OR (obj:Computer))
CALL apoc.create.addLabels(obj, ["Tier" + o.tier]) YIELD node
RETURN null
'''

tier_users = '''
UNWIND value.users as u
MATCH (user:User) WHERE user.objectid ENDS WITH u.RID
CALL apoc.create.addLabels(user, ["Tier" + u.tier]) YIELD node
RETURN null
'''

tier_groups = '''
UNWIND value.groups as g
MATCH (group:Group) WHERE group.objectid ENDS WITH g.RID
WITH group, g
MATCH (principal)-[:MemberOf*1..]->(group) WHERE (principal:User) OR (principal:Computer)
CALL apoc.create.addLabels(principal, ["Tier" + g.tier]) YIELD node
RETURN null
'''

Tier the data

In [11]:
# Generic tiering
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_groups)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_users)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_ous)
out = conn.query(query)


# Customer specific tiering
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_groups)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_users)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_ous)
out = conn.query(query)

# Delete higher tier labels for objects in multiple tiers
query = '''
match (n) 
UNWIND labels(n) as label
WITH n, label WHERE label STARTS WITH "Tier"
WITH n, label ORDER BY label ASC
WITH n, size(collect(label)) as noOfTiers, collect(label) as tiers
CALL apoc.create.removeLabels(n, tiers[1..noOfTiers])
YIELD node RETURN null
'''
out = conn.query(query)

Get names of objects in a given tier

In [56]:
query = '''
MATCH (obj:Tier0) RETURN obj.name LIMIT 10
'''
out = conn.query(query)

Get number of principals in a given tier

In [58]:
query = '''
MATCH (obj:Tier0) RETURN COUNT(obj)
'''
out = conn.query(query)

Get number of Tier 2 principals with path to Tier 0 principals

In [ ]:
query = '''MATCH (t0:Tier0)
CALL apoc.path.expandConfig(t0, {
    relationshipFilter: "<",
    labelFilter: ">Tier2",
    minLevel: 1,
    maxLevel: 3
})
YIELD path RETURN path
'''

Clean graph from Tier nodes and relations

In [10]:
query = '''
CALL db.labels()
YIELD label WHERE label STARTS WITH "Tier"
WITH collect(label) AS labels
MATCH (p)
WITH collect(p) AS people, labels
CALL apoc.create.removeLabels(people, labels)
YIELD node RETURN null
'''
out = conn.query(query)